In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 45.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=c919f82a4aeaaba1515afbcb9d7164789f76e8675dde7fb5923ab3e91aa5a210
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark import SparkConf,SparkContext
#sparkconf is a object,conf is aa spark application,it is used to set various parameters as key-value pair
#sparkcontext it is connection to spark cluster,used to create a RDD and broadcast in that cluster
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('flight').getOrCreate()#appName=applicationName
from pyspark.sql import SQLContext
#SQLContext is use of main entry point for DataFrame and SQL functionality
SQLContext = SQLContext(spark)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/feb-20-us-flight-delay.csv')

In [ ]:
df

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,ORIGIN,DEST,DEP_TIME,DEP_DEL15,DISTANCE
0,2,1,6,MQ,CLT,LYH,1430.0,0.0,175
1,2,8,6,MQ,CLT,LYH,1442.0,0.0,175
2,2,13,4,MQ,DFW,SHV,2247.0,0.0,190
3,2,14,5,MQ,DFW,SHV,2230.0,0.0,190
4,2,15,6,MQ,DFW,SHV,2246.0,0.0,190
...,...,...,...,...,...,...,...,...,...
574263,2,27,4,DL,ATL,SRQ,2226.0,0.0,444
574264,2,27,4,DL,ATL,BWI,737.0,0.0,577
574265,2,27,4,DL,BWI,ATL,1026.0,0.0,577
574266,2,27,4,DL,MSP,ATL,1008.0,0.0,907




1. MONTH - Month
2. DAY_OF_MONTH - Day of Month
3. DAY_OF_WEEK - Day of Week
4. OP_UNIQUE_CARRIER - Unique Carrier Code
5. ORIGIN - Origin airport location
6. DEST - Destination airport location
7. DEP_TIME - Actual Departure Time (local time: hhmm)
8. DEP_DEL15 - Departure Delay Indicator, 15 Minutes or More (1=Yes, 0=No) [TARGET VARIABLE]
9. DISTANCE - Distance between airports (miles)


In [ ]:
df.isnull().sum()

MONTH                   0
DAY_OF_MONTH            0
DAY_OF_WEEK             0
OP_UNIQUE_CARRIER       0
ORIGIN                  0
DEST                    0
DEP_TIME             4938
DEP_DEL15            4951
DISTANCE                0
dtype: int64

In [ ]:
df.dropna()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,ORIGIN,DEST,DEP_TIME,DEP_DEL15,DISTANCE
0,2,1,6,MQ,CLT,LYH,1430.0,0.0,175
1,2,8,6,MQ,CLT,LYH,1442.0,0.0,175
2,2,13,4,MQ,DFW,SHV,2247.0,0.0,190
3,2,14,5,MQ,DFW,SHV,2230.0,0.0,190
4,2,15,6,MQ,DFW,SHV,2246.0,0.0,190
...,...,...,...,...,...,...,...,...,...
574263,2,27,4,DL,ATL,SRQ,2226.0,0.0,444
574264,2,27,4,DL,ATL,BWI,737.0,0.0,577
574265,2,27,4,DL,BWI,ATL,1026.0,0.0,577
574266,2,27,4,DL,MSP,ATL,1008.0,0.0,907


In [ ]:
flight=spark.read.csv('/content/drive/MyDrive/feb-20-us-flight-delay.csv',inferSchema=True,header=True)

In [ ]:
flight.count()

574268

In [ ]:
flight.describe().show()

+-------+------+-----------------+------------------+-----------------+------+------+------------------+-------------------+-----------------+
|summary| MONTH|     DAY_OF_MONTH|       DAY_OF_WEEK|OP_UNIQUE_CARRIER|ORIGIN|  DEST|          DEP_TIME|          DEP_DEL15|         DISTANCE|
+-------+------+-----------------+------------------+-----------------+------+------+------------------+-------------------+-----------------+
|  count|574268|           574268|            574268|           574268|574268|574268|            569330|             569317|           574268|
|   mean|   2.0|  15.179477177903|3.9892802663564746|             null|  null|  null|1332.6079057839918|0.14435191641914785|795.7355224390006|
| stddev|   0.0|8.295999386744485|1.9883451440613822|             null|  null|  null| 498.6673040551802|0.35144652167522156|582.9608167381957|
|    min|     2|                1|                 1|               9E|   ABE|   ABE|                 1|                  0|               31|

In [ ]:
flight.groupBy('DISTANCE').count().show()

+--------+-----+
|DISTANCE|count|
+--------+-----+
|     463|  485|
|     833|  508|
|    1829|  116|
|    1342|  202|
|     148|  250|
|     471|  274|
|    1959|   58|
|     496|  592|
|    1088|   58|
|    1507|   54|
|     737|  729|
|    1721|  658|
|     623|  442|
|     392|  150|
|    1460|  317|
|    1025|  260|
|    1127|  538|
|    1084|  501|
|     243|   98|
|    1522|  143|
+--------+-----+
only showing top 20 rows



In [ ]:
# create DataFrame as a temporary view
tempview = flight.createOrReplaceTempView('table')
tempview

In [ ]:
spark.sql("SELECT DAY_OF_MONTH, count(DAY_OF_MONTH) as count_DAY_OF_MONTH, count(DAY_OF_MONTH)*100/sum(count(DAY_OF_MONTH)) over() as percent FROM table GROUP BY DAY_OF_MONTH").show()

+------------+------------------+------------------+
|DAY_OF_MONTH|count_DAY_OF_MONTH|           percent|
+------------+------------------+------------------+
|          28|             21309| 3.710636845514638|
|          26|             20668| 3.599016487075721|
|          27|             21286| 3.706631746849903|
|          12|             19951| 3.474161889570723|
|          22|             17111| 2.979619271838236|
|           1|             15780| 2.747845953457271|
|          13|             21185|  3.68904413966998|
|          16|             18902|3.2914945635138992|
|           6|             20826| 3.626529773555204|
|           3|             20708|3.6059818760578684|
|          20|             21270|3.7038455912570436|
|           5|             19775| 3.443514178049273|
|          19|             20661| 3.597797544003845|
|          15|             16678|2.9042189361064867|
|          17|             20989| 3.654913733657456|
|           9|             20007|  3.483913434

In [ ]:
spark.sql("SELECT DAY_OF_WEEK, count(DAY_OF_WEEK) as DAY_OF_WEEK FROM table WHERE DAY_OF_WEEK <= 50 GROUP BY DAY_OF_WEEK ORDER BY DAY_OF_WEEK DESC").show()

+-----------+-----------+
|DAY_OF_WEEK|DAY_OF_WEEK|
+-----------+-----------+
|          7|      78150|
|          6|      82720|
|          5|      84738|
|          4|      84567|
|          3|      81055|
|          2|      79385|
|          1|      83653|
+-----------+-----------+



In [ ]:
flight.show()

+-----+------------+-----------+-----------------+------+----+--------+---------+--------+
|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|ORIGIN|DEST|DEP_TIME|DEP_DEL15|DISTANCE|
+-----+------------+-----------+-----------------+------+----+--------+---------+--------+
|    2|           1|          6|               MQ|   CLT| LYH|    1430|        0|     175|
|    2|           8|          6|               MQ|   CLT| LYH|    1442|        0|     175|
|    2|          13|          4|               MQ|   DFW| SHV|    2247|        0|     190|
|    2|          14|          5|               MQ|   DFW| SHV|    2230|        0|     190|
|    2|          15|          6|               MQ|   DFW| SHV|    2246|        0|     190|
|    2|          16|          7|               MQ|   DFW| SHV|    2230|        0|     190|
|    2|          17|          1|               MQ|   DFW| SHV|    2237|        0|     190|
|    2|          18|          2|               MQ|   DFW| SHV|    2230|        0|     190|

In [ ]:
#categorical features convert into numeric values 
#so used stringindexer
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCols=["OP_UNIQUE_CARRIER","ORIGIN","DEST"], outputCols=["OP_UNIQUE_CARRIER_Encode","ORIGIN_Encode","DEST_Encode"])
indexed = indexer.fit(flight).transform(flight)#fit a model with input dataset with optinal parameter
indexed.show() 

+-----+------------+-----------+-----------------+------+----+--------+---------+--------+------------------------+-------------+-----------+
|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|ORIGIN|DEST|DEP_TIME|DEP_DEL15|DISTANCE|OP_UNIQUE_CARRIER_Encode|ORIGIN_Encode|DEST_Encode|
+-----+------------+-----------+-----------------+------+----+--------+---------+--------+------------------------+-------------+-----------+
|    2|           1|          6|               MQ|   CLT| LYH|    1430|        0|     175|                     6.0|          4.0|      246.0|
|    2|           8|          6|               MQ|   CLT| LYH|    1442|        0|     175|                     6.0|          4.0|      246.0|
|    2|          13|          4|               MQ|   DFW| SHV|    2247|        0|     190|                     6.0|          2.0|      130.0|
|    2|          14|          5|               MQ|   DFW| SHV|    2230|        0|     190|                     6.0|          2.0|      130.0|
|    2

In [ ]:
indexed=indexed.drop("OP_UNIQUE_CARRIER","ORIGIN","DEST")

In [ ]:
indexed.show()

+-----+------------+-----------+--------+---------+--------+------------------------+-------------+-----------+
|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|DEP_TIME|DEP_DEL15|DISTANCE|OP_UNIQUE_CARRIER_Encode|ORIGIN_Encode|DEST_Encode|
+-----+------------+-----------+--------+---------+--------+------------------------+-------------+-----------+
|    2|           1|          6|    1430|        0|     175|                     6.0|          4.0|      246.0|
|    2|           8|          6|    1442|        0|     175|                     6.0|          4.0|      246.0|
|    2|          13|          4|    2247|        0|     190|                     6.0|          2.0|      130.0|
|    2|          14|          5|    2230|        0|     190|                     6.0|          2.0|      130.0|
|    2|          15|          6|    2246|        0|     190|                     6.0|          2.0|      130.0|
|    2|          16|          7|    2230|        0|     190|                     6.0|          2.0|     

In [ ]:
indexed.na.drop(how='any').show()

+-----+------------+-----------+--------+---------+--------+------------------------+-------------+-----------+
|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|DEP_TIME|DEP_DEL15|DISTANCE|OP_UNIQUE_CARRIER_Encode|ORIGIN_Encode|DEST_Encode|
+-----+------------+-----------+--------+---------+--------+------------------------+-------------+-----------+
|    2|           1|          6|    1430|        0|     175|                     6.0|          4.0|      246.0|
|    2|           8|          6|    1442|        0|     175|                     6.0|          4.0|      246.0|
|    2|          13|          4|    2247|        0|     190|                     6.0|          2.0|      130.0|
|    2|          14|          5|    2230|        0|     190|                     6.0|          2.0|      130.0|
|    2|          15|          6|    2246|        0|     190|                     6.0|          2.0|      130.0|
|    2|          16|          7|    2230|        0|     190|                     6.0|          2.0|     

In [ ]:
indexed.show()

+-----+------------+-----------+--------+---------+--------+------------------------+-------------+-----------+
|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|DEP_TIME|DEP_DEL15|DISTANCE|OP_UNIQUE_CARRIER_Encode|ORIGIN_Encode|DEST_Encode|
+-----+------------+-----------+--------+---------+--------+------------------------+-------------+-----------+
|    2|           1|          6|    1430|        0|     175|                     6.0|          4.0|      246.0|
|    2|           8|          6|    1442|        0|     175|                     6.0|          4.0|      246.0|
|    2|          13|          4|    2247|        0|     190|                     6.0|          2.0|      130.0|
|    2|          14|          5|    2230|        0|     190|                     6.0|          2.0|      130.0|
|    2|          15|          6|    2246|        0|     190|                     6.0|          2.0|      130.0|
|    2|          16|          7|    2230|        0|     190|                     6.0|          2.0|     

In [ ]:
#null values
indexed = indexed.na.drop(how='any')

In [ ]:
indexed.show()

+-----+------------+-----------+--------+---------+--------+------------------------+-------------+-----------+
|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|DEP_TIME|DEP_DEL15|DISTANCE|OP_UNIQUE_CARRIER_Encode|ORIGIN_Encode|DEST_Encode|
+-----+------------+-----------+--------+---------+--------+------------------------+-------------+-----------+
|    2|           1|          6|    1430|        0|     175|                     6.0|          4.0|      246.0|
|    2|           8|          6|    1442|        0|     175|                     6.0|          4.0|      246.0|
|    2|          13|          4|    2247|        0|     190|                     6.0|          2.0|      130.0|
|    2|          14|          5|    2230|        0|     190|                     6.0|          2.0|      130.0|
|    2|          15|          6|    2246|        0|     190|                     6.0|          2.0|      130.0|
|    2|          16|          7|    2230|        0|     190|                     6.0|          2.0|     

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(
    inputCols=['MONTH','DAY_OF_MONTH','DAY_OF_WEEK','DEP_TIME','DISTANCE','OP_UNIQUE_CARRIER_Encode','ORIGIN_Encode'],
         outputCol="features")       

In [ ]:
output = assembler.transform(indexed)

In [ ]:
final_data = output.select("features","DEP_DEL15")

In [ ]:
final_data.show()

+--------------------+---------+
|            features|DEP_DEL15|
+--------------------+---------+
|[2.0,1.0,6.0,1430...|        0|
|[2.0,8.0,6.0,1442...|        0|
|[2.0,13.0,4.0,224...|        0|
|[2.0,14.0,5.0,223...|        0|
|[2.0,15.0,6.0,224...|        0|
|[2.0,16.0,7.0,223...|        0|
|[2.0,17.0,1.0,223...|        0|
|[2.0,18.0,2.0,223...|        0|
|[2.0,19.0,3.0,224...|        0|
|[2.0,20.0,4.0,222...|        0|
|[2.0,21.0,5.0,223...|        0|
|[2.0,22.0,6.0,223...|        0|
|[2.0,23.0,7.0,223...|        0|
|[2.0,24.0,1.0,223...|        0|
|[2.0,25.0,2.0,223...|        0|
|[2.0,26.0,3.0,222...|        0|
|[2.0,27.0,4.0,222...|        0|
|[2.0,28.0,5.0,222...|        0|
|[2.0,29.0,6.0,223...|        0|
|[2.0,1.0,6.0,1557...|        0|
+--------------------+---------+
only showing top 20 rows



In [ ]:
final_data = output.select("features","DEP_DEL15")
train_data, test_data = final_data.randomSplit([0.7,0.3])
#randomly i split my train and test data random size

In [ ]:
final_data.show()

+--------------------+---------+
|            features|DEP_DEL15|
+--------------------+---------+
|[2.0,1.0,6.0,1430...|        0|
|[2.0,8.0,6.0,1442...|        0|
|[2.0,13.0,4.0,224...|        0|
|[2.0,14.0,5.0,223...|        0|
|[2.0,15.0,6.0,224...|        0|
|[2.0,16.0,7.0,223...|        0|
|[2.0,17.0,1.0,223...|        0|
|[2.0,18.0,2.0,223...|        0|
|[2.0,19.0,3.0,224...|        0|
|[2.0,20.0,4.0,222...|        0|
|[2.0,21.0,5.0,223...|        0|
|[2.0,22.0,6.0,223...|        0|
|[2.0,23.0,7.0,223...|        0|
|[2.0,24.0,1.0,223...|        0|
|[2.0,25.0,2.0,223...|        0|
|[2.0,26.0,3.0,222...|        0|
|[2.0,27.0,4.0,222...|        0|
|[2.0,28.0,5.0,222...|        0|
|[2.0,29.0,6.0,223...|        0|
|[2.0,1.0,6.0,1557...|        0|
+--------------------+---------+
only showing top 20 rows



In [ ]:
df = final_data

In [ ]:
type(df.rdd)

pyspark.rdd.RDD

In [ ]:
df.show()

+--------------------+---------+
|            features|DEP_DEL15|
+--------------------+---------+
|[2.0,1.0,6.0,1430...|        0|
|[2.0,8.0,6.0,1442...|        0|
|[2.0,13.0,4.0,224...|        0|
|[2.0,14.0,5.0,223...|        0|
|[2.0,15.0,6.0,224...|        0|
|[2.0,16.0,7.0,223...|        0|
|[2.0,17.0,1.0,223...|        0|
|[2.0,18.0,2.0,223...|        0|
|[2.0,19.0,3.0,224...|        0|
|[2.0,20.0,4.0,222...|        0|
|[2.0,21.0,5.0,223...|        0|
|[2.0,22.0,6.0,223...|        0|
|[2.0,23.0,7.0,223...|        0|
|[2.0,24.0,1.0,223...|        0|
|[2.0,25.0,2.0,223...|        0|
|[2.0,26.0,3.0,222...|        0|
|[2.0,27.0,4.0,222...|        0|
|[2.0,28.0,5.0,222...|        0|
|[2.0,29.0,6.0,223...|        0|
|[2.0,1.0,6.0,1557...|        0|
+--------------------+---------+
only showing top 20 rows



In [ ]:
train_data.show()

+--------------------+---------+
|            features|DEP_DEL15|
+--------------------+---------+
|[2.0,1.0,6.0,3.0,...|        0|
|[2.0,1.0,6.0,4.0,...|        0|
|[2.0,1.0,6.0,4.0,...|        0|
|[2.0,1.0,6.0,9.0,...|        1|
|[2.0,1.0,6.0,10.0...|        0|
|[2.0,1.0,6.0,15.0...|        1|
|[2.0,1.0,6.0,15.0...|        0|
|[2.0,1.0,6.0,17.0...|        1|
|[2.0,1.0,6.0,18.0...|        0|
|[2.0,1.0,6.0,23.0...|        0|
|[2.0,1.0,6.0,25.0...|        0|
|[2.0,1.0,6.0,31.0...|        0|
|[2.0,1.0,6.0,32.0...|        0|
|[2.0,1.0,6.0,33.0...|        0|
|[2.0,1.0,6.0,34.0...|        1|
|[2.0,1.0,6.0,35.0...|        0|
|[2.0,1.0,6.0,101....|        0|
|[2.0,1.0,6.0,106....|        0|
|[2.0,1.0,6.0,112....|        1|
|[2.0,1.0,6.0,115....|        0|
+--------------------+---------+
only showing top 20 rows



In [ ]:
train_data.describe().show()

+-------+-------------------+
|summary|          DEP_DEL15|
+-------+-------------------+
|  count|             397438|
|   mean|0.14402497999688002|
| stddev|0.35111550140345027|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [ ]:
test_data.describe().show()

+-------+-------------------+
|summary|          DEP_DEL15|
+-------+-------------------+
|  count|             171879|
|   mean|0.14510789567079166|
| stddev| 0.3522106131662122|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [ ]:
#create linear regression model using logistic reegression 
#it is s classification model 
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol = "DEP_DEL15")
#fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data)

In [ ]:
result = lrModel.evaluate(train_data)

In [ ]:
pred = lrModel.transform(train_data).show()

+--------------------+---------+--------------------+--------------------+----------+
|            features|DEP_DEL15|       rawPrediction|         probability|prediction|
+--------------------+---------+--------------------+--------------------+----------+
|[2.0,1.0,6.0,3.0,...|        0|[2.98602300166137...|[0.95193868349121...|       0.0|
|[2.0,1.0,6.0,4.0,...|        0|[2.97429041385035...|[0.95139904567304...|       0.0|
|[2.0,1.0,6.0,4.0,...|        0|[2.99746469488055...|[0.95245945869540...|       0.0|
|[2.0,1.0,6.0,9.0,...|        1|[2.81842450356045...|[0.94366336712074...|       0.0|
|[2.0,1.0,6.0,10.0...|        0|[2.89651188729405...|[0.94767373745623...|       0.0|
|[2.0,1.0,6.0,15.0...|        1|[2.90234828014699...|[0.94796239878333...|       0.0|
|[2.0,1.0,6.0,15.0...|        0|[2.90932890411989...|[0.94830567595361...|       0.0|
|[2.0,1.0,6.0,17.0...|        1|[2.72615771892661...|[0.93855261971690...|       0.0|
|[2.0,1.0,6.0,18.0...|        0|[2.96303092032799...|[

In [ ]:
#evaluate test data
result = lrModel.evaluate(test_data)

In [ ]:
#accuracy score
result.accuracy

0.8548921043292084

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(labelCol='DEP_DEL15')
#fit the model to the data and call this model lrModel
dtModel = dt.fit(train_data)

In [ ]:
result.accuracy

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol='DEP_DEL15')
rfModel = dt.fit(train_data)

In [ ]:
result.accuracy